In [ ]:
# !pip install autogen pandas matplotlib seaborn
#!pip install google-generativeai
#!pip install ag2[gemini]

### PLease check the output 5th cell. The code kept running so I have interruped the kernel for the time being otherwise I will run out of Gemini-2.0-flash API limits.

In [2]:
import autogen
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import os

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Ensure output directory exists
OUTPUT_DIR = "eda_reports"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Configuration ---
config_list_gpt = autogen.config_list_from_json(
    env_or_file="OAI_CONFIG_LIST.json",
    filter_dict={"model": ["gemini-2.0-flash"]},
)

C:\Users\krish\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\krish\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\krish\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [5]:
# --- Define Helper Functions ---
def save_report(report_content, filename="eda_report.md"):
    filepath = os.path.join(OUTPUT_DIR, filename)
    with open(filepath, "w") as f:
        f.write(report_content)
    logger.info(f"EDA Report saved to: {filepath}")
    return filepath

def visualize_data(df, filename_prefix="eda_visualization"):
    image_paths = []
    os.makedirs(os.path.join(OUTPUT_DIR, "visualizations"), exist_ok=True)

    try:
        # Histograms for numerical features
        numerical_cols = df.select_dtypes(include=['number']).columns
        for col in numerical_cols:
            plt.figure(figsize=(8, 6))
            sns.histplot(df[col], kde=True)
            plt.title(f'Distribution of {col}')
            filepath = os.path.join(OUTPUT_DIR, "visualizations", f"{filename_prefix}_histogram_{col}.png")
            plt.savefig(filepath)
            plt.close()
            image_paths.append(filepath)

        # Box plots for numerical features
        for col in numerical_cols:
            plt.figure(figsize=(8, 6))
            sns.boxplot(x=df[col])
            plt.title(f'Box Plot of {col}')
            filepath = os.path.join(OUTPUT_DIR, "visualizations", f"{filename_prefix}_boxplot_{col}.png")
            plt.savefig(filepath)
            plt.close()
            image_paths.append(filepath)

        # Correlation heatmap if multiple numerical features exist
        if len(numerical_cols) > 1:
            plt.figure(figsize=(10, 8))
            sns.heatmap(df[numerical_cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
            plt.title('Correlation Heatmap')
            filepath = os.path.join(OUTPUT_DIR, "visualizations", f"{filename_prefix}_correlation.png")
            plt.savefig(filepath)
            plt.close()
            image_paths.append(filepath)

        logger.info(f"Visualizations saved to: {os.path.join(OUTPUT_DIR, 'visualizations')}")
    except Exception as e:
        logger.error(f"Error during visualization: {e}")
    return image_paths

# --- Define Agents ---

# Data Preparation Agent
data_preparation_agent = autogen.AssistantAgent(
    name="DataPreparationAgent",
    llm_config={"config_list": config_list_gpt, "seed": 42},
    system_message="You are responsible for cleaning and preprocessing the raw input data. Your tasks include handling missing values, converting data types, and preparing the data for exploratory analysis. You will output a Python script that loads and preprocesses the data, saving the processed DataFrame to a CSV file.",
)

# EDA Agent
eda_agent = autogen.AssistantAgent(
    name="EDAAgent",
    llm_config={"config_list": config_list_gpt, "seed": 42},
    system_message="You are an expert in Exploratory Data Analysis. You will receive a processed DataFrame and your goal is to perform EDA. This includes generating statistical summaries, identifying patterns, and creating relevant visualizations using pandas, matplotlib, and seaborn. Output Python code to perform these tasks and save the visualizations.",
)

# Report Generator Agent
report_generator_agent = autogen.AssistantAgent(
    name="ReportGeneratorAgent",
    llm_config={"config_list": config_list_gpt, "seed": 42},
    system_message="You are responsible for compiling a comprehensive EDA report. This report should include a dataset overview, key insights discovered during EDA, and the generated visualizations. Present the findings in a clear and concise manner, suitable for stakeholders. You will receive the EDA findings and the paths to the generated image files. Output the report in Markdown format.",
)

# Critic Agent
critic_agent = autogen.AssistantAgent(
    name="CriticAgent",
    llm_config={"config_list": config_list_gpt, "seed": 42},
    system_message="You are a critical reviewer. You will receive the EDA report and your task is to review it for clarity, accuracy, completeness, and relevance. Provide constructive feedback and suggest improvements.",
)

# Executor Agent
executor_agent = autogen.UserProxyAgent(
    name="ExecutorAgent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    llm_config={"config_list": config_list_gpt, "seed": 42},
    code_execution_config={"work_dir": "code", "use_docker": False},  # Set use_docker to True if you have Docker
)

# Admin Agent
admin_agent = autogen.ConversableAgent(
    name="AdminAgent",
    llm_config={"config_list": config_list_gpt, "seed": 42},
    system_message="You are the administrator overseeing the multi-agent EDA workflow. You will initiate tasks, coordinate between agents, and ensure the process aligns with the project goals. Guide the agents through data preparation, EDA, report generation, and critique.",
)

# --- Workflow Implementation ---

def run_eda_pipeline(csv_filepath):
    try:
        df = pd.read_csv(csv_filepath)
        logger.info(f"Successfully loaded data from: {csv_filepath}")
    except FileNotFoundError:
        logger.error(f"Error: CSV file not found at {csv_filepath}")
        return None

    # 1. Data Preparation
    admin_agent.initiate_chat(
        data_preparation_agent,
        message=f"Load and preprocess the data from the CSV file: {csv_filepath}. Output a Python script to do this, and save the processed data to 'processed_data.csv'.",
    )
    preparation_code = executor_agent.last_code_info["content"] if executor_agent.last_code_info else None
    if preparation_code:
        logger.info("Executing data preparation code.")
        executor_agent.execute_code(preparation_code)
        try:
            processed_df = pd.read_csv("code/processed_data.csv")
            logger.info("Data preparation successful.")
        except FileNotFoundError:
            logger.error("Error: 'processed_data.csv' not found after data preparation.")
            return None
    else:
        logger.warning("No data preparation code was generated.")
        processed_df = df.copy() # Proceed with original if no prep code

    # 2. Exploratory Data Analysis
    admin_agent.send(
        message="Now, perform exploratory data analysis on the processed data. Generate Python code to get statistical summaries and create relevant visualizations (histograms, box plots, correlation heatmap if applicable). Save the visualizations as image files.",
        recipient=eda_agent,
    )
    eda_code = executor_agent.last_code_info["content"] if executor_agent.last_code_info else None
    image_paths = []
    if eda_code:
        logger.info("Executing EDA code.")
        executor_agent.execute_code(eda_code)
        image_paths = visualize_data(processed_df)
    else:
        logger.warning("No EDA code was generated. Proceeding with basic visualization.")
        image_paths = visualize_data(processed_df)

    # 3. Report Generation (Initial)
    eda_insights = admin_agent.last_message(eda_agent) if eda_agent.last_message() else "No specific EDA insights generated."
    admin_agent.send(
        message=f"Generate an initial EDA report based on the dataset overview, the following insights: '{eda_insights}', and include the visualizations from these file paths: {image_paths}. The report should be in Markdown format.",
        recipient=report_generator_agent,
    )
    initial_report = report_generator_agent.last_message()

    # 4. Critique
    admin_agent.send(
        message=f"Review the following EDA report and provide constructive feedback for improvement: \n\n{initial_report}",
        recipient=critic_agent,
    )
    critique = critic_agent.last_message()

    # 5. Report Revision
    admin_agent.send(
        message=f"Based on the following feedback: '{critique}', revise the EDA report to address the suggestions. Ensure the final report is clear, accurate, and includes the visualizations.",
        recipient=report_generator_agent,
    )
    revised_report = report_generator_agent.last_message()

    # Save the final report
    if revised_report:
        report_filepath = save_report(revised_report.content, "final_eda_report.md")
        logger.info(f"Final EDA report generated at: {report_filepath}")
        return report_filepath
    elif initial_report:
        report_filepath = save_report(initial_report.content, "final_eda_report.md")
        logger.warning("No revised report generated. Saving the initial report.")
        return report_filepath
    else:
        logger.error("No EDA report could be generated.")
        return None

if __name__ == "__main__":
    # Update the agent configurations to use config_list_gemini
    # llm_config_gemini = {"config_list": config_list_gemini, "seed": 42}
    
    # data_preparation_agent.llm_config = llm_config_gemini
    # eda_agent.llm_config = llm_config_gemini
    # report_generator_agent.llm_config = llm_config_gemini
    # critic_agent.llm_config = llm_config_gemini
    # executor_agent.llm_config = llm_config_gemini
    # admin_agent.llm_config = llm_config_gemini

    # Create a sample CSV file for testing
    sample_data = {
        'feature1': [1, 2, 3, 4, 5, None],
        'feature2': [5.5, 4.3, 7.1, 6.2, 8.0, 5.1],
        'category': ['A', 'B', 'A', 'C', 'B', 'A']
    }
    sample_df = pd.DataFrame(sample_data)
    sample_csv_path = "sample_data.csv"
    sample_df.to_csv(sample_csv_path, index=False)
    print(f"Sample data saved to: {sample_csv_path}")

    report_file = run_eda_pipeline(sample_csv_path)
    if report_file:
        print(f"\nEDA pipeline completed. Report available at: {report_file}")
    else:
        print("\nEDA pipeline failed to generate a report.")

2025-06-03 12:29:18,112 - __main__ - INFO - Successfully loaded data from: sample_data.csv


Sample data saved to: sample_data.csv
AdminAgent (to DataPreparationAgent):

Load and preprocess the data from the CSV file: sample_data.csv. Output a Python script to do this, and save the processed data to 'processed_data.csv'.

--------------------------------------------------------------------------------


2025-06-03 12:29:18,421 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-06-03 12:29:22,776 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-06-03 12:29:22,797 - google_genai.models - INFO - AFC remote call 1 is done.


DataPreparationAgent (to AdminAgent):

```python
import pandas as pd
import numpy as np

def preprocess_data(input_csv, output_csv):
    """
    Loads, preprocesses, and saves data from a CSV file.

    Args:
        input_csv (str): Path to the input CSV file.
        output_csv (str): Path to save the processed CSV file.
    """

    try:
        # Load the data
        df = pd.read_csv(input_csv)

        # Handle missing values (replace with mean for numerical, mode for categorical)
        for col in df.columns:
            if df[col].isnull().any():
                if pd.api.types.is_numeric_dtype(df[col]):
                    df[col] = df[col].fillna(df[col].mean())
                else:
                    df[col] = df[col].fillna(df[col].mode()[0])

        # Convert data types (if necessary) - Example: Convert 'Date' column to datetime
        # Example assumes there is a Date column, if it doesn't exist, this will raise a KeyError
        if 'Date' in df.columns:
           

2025-06-03 12:29:23,142 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-06-03 12:29:25,284 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-06-03 12:29:25,304 - google_genai.models - INFO - AFC remote call 1 is done.


...

AdminAgent (to DataPreparationAgent):

Yes, let's tackle another task. This time, we'll focus on Natural Language Processing (NLP).

I have a text file named `news_articles.txt` containing a collection of news articles. The task is to perform topic modeling using Latent Dirichlet Allocation (LDA).

Here's the task:

1.  **Data Preparation:** Load the text data from `news_articles.txt`. Perform text cleaning steps such as removing punctuation, converting text to lowercase, and removing stop words.

2.  **Topic Modeling:** Apply LDA to discover the top 5 topics present in the news articles. Use scikit-learn's `LatentDirichletAllocation` class.

3.  **Topic Interpretation:** For each topic, identify the top 10 most frequent words and provide a brief interpretation of the topic based on these words.

Output a Python script that accomplishes these tasks and saves the topic interpretations (topic number, top words, and interpretation) to a text file named `topic_interpretations.txt`.


-----

2025-06-03 12:31:05,929 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-06-03 12:31:12,584 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-06-03 12:31:12,584 - google_genai.models - INFO - AFC remote call 1 is done.


DataPreparationAgent (to AdminAgent):

```python
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def perform_topic_modeling(input_file, output_file, num_topics=5, top_words=10):
    """
    Performs topic modeling using LDA on news articles and saves topic interpretations.

    Args:
        input_file (str): Path to the input text file containing news articles.
        output_file (str): Path to save the topic interpretations.
        num_topics (int): Number of topics to discover.
        top_words (int): Number of top words to identify for each topic.
    """

    try:
        # 1. Data Preparation
        with open(input_file, 'r', encoding='utf-8') as f:
            text_data = f.read()

        # Tokenize the text data into individual articles.  Assumes each article is separated by a newline
        articles =

2025-06-03 12:31:12,912 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.


KeyboardInterrupt: 